# Odoo y JSON-RPC

Este notebook muestra como acceder a **Odoo** utilizando la interfaz **JSON-RPC** utilizando solamente las bibliotecas `json` y `urllib`.

In [1]:
host = "192.168.86.131"

In [2]:
port = 8069

In [3]:
url = "http://{}:{}/jsonrpc".format(host,port)

In [4]:
db ="prueba"

In [5]:
user = "admin@odoo3.com"

In [6]:
password = "prueba"

In [7]:
import json

In [8]:
import random

In [9]:
from urllib import request

In [10]:
def json_rpc(url, method, params):
    data = {
        "jsonrpc": "2.0",
        "method": method,
        "params": params,
        "id": random.randint(0, 1000000000),
    }
    req = request.Request(url, data=json.dumps(data).encode(), headers={
        "Content-Type":"application/json",
    })
    reply = json.load(request.urlopen(req))
    if reply.get("error"):
        raise Exception(reply["error"])
    return reply["result"]

In [11]:
def call(url, service, method, *args):
    return json_rpc(url, "call", {"service": service, "method": method, "args": args})

Tenemos que autenticarnos para acceder al servidor. Una llamada a *login* devuelve el *id* del usuario, que necesitaremos para ejecutar llamadas a los objetos con *execute*

In [12]:
uid = call(url, "common", "login", db, user, password)

In [13]:
args = {
    'color' : 8,
    'memo' : 'This is another note',
    'create_uid': uid,
}

In [14]:
note_id = call(url, "object", "execute", db, uid, password, 'note.note', 'create', args)

In [15]:
call(url,"object","execute",
     db, uid, password, "note.note","search_read",
     [["id","=",note_id]])

[{'__last_update': '2017-11-09 02:44:34',
  'color': 8,
  'create_date': '2017-11-09 02:44:34',
  'create_uid': [1, 'Administrator'],
  'date_done': False,
  'display_name': 'This is another note',
  'id': 12,
  'memo': '<p>This is another note</p>',
  'message_channel_ids': [],
  'message_follower_ids': [18],
  'message_ids': [51, 50],
  'message_is_follower': True,
  'message_last_post': False,
  'message_needaction': False,
  'message_needaction_counter': 0,
  'message_partner_ids': [3],
  'message_unread': False,
  'message_unread_counter': 0,
  'name': 'This is another note',
  'open': True,
  'sequence': 0,
  'stage_id': [1, 'New'],
  'stage_ids': [1],
  'tag_ids': [],
  'user_id': [1, 'Administrator'],
  'write_date': '2017-11-09 02:44:34',
  'write_uid': [1, 'Administrator']}]